<a href="https://colab.research.google.com/github/damsdaddy/BDC101/blob/main/NSMC_%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Step1. 라이브러리를 로드합니다.

In [1]:
!pip install chatspace

     |████████████████████████████████| 1.2MB 8.2MB/s 


In [12]:
import pandas as pd
import numpy as np
import re, json
from bs4 import BeautifulSoup
import urllib.request
from urllib.error import HTTPError
import chatspace
spacer = chatspace.Chatspace()
from tqdm.notebook import tqdm
tqdm.pandas()

Step2. 구글 드라이브를 마운트합니다.

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Step3. 파일을 불러옵니다.

In [5]:
mypath = '/content/drive/MyDrive/Colab Notebooks/dataset/' # 구글 드라이브에 파일이 있는 폴더경로를 설정
train_data = pd.read_table(mypath+'ratings_train.txt')
test_data = pd.read_table(mypath+'ratings_test.txt')

Step4. 데이터가 정상적으로 로드되었는지를 확인합니다.

In [6]:
train_data[:5] # 상위 5개 출력

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [7]:
test_data[:5]

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


Step5. 데이터를 전처리합니다.
- 휴리스틱 규칙으로 대표적인 비표준어들을 통일시킵니다.
- 한글숫자공백외 문자를 제거합니다.
- null 행을 제거합니다.
- 중복행을 제거합니다.

In [8]:
# train_data 전처리

# 휴리스틱 검출규칙 : 비표준어에 대해 직관적인 방법으로 규칙을 만들어 교체
heuristic_rule = [
    ['ㅋㅋ+', ' ㅋㅋ '],
    ['ㄹㄹ+', ' ㄹㄹ '],
    ['ㅜㅜ+', ' ㅜㅜ '],
    ['ㅠㅠ+', ' ㅠㅠ '],
    ['ㄷㄷ+', ' ㄷㄷ '],
    ['ㅎㅎ+', ' ㅎㅎ '],
    ['ㅡㅡ+', ' ㅡㅡ '],
    ['^ +', ''],    
    ['  +', ' ']
]

train_data['document'] = train_data['document'].str.replace("[^0-9ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 한글숫자공백외 문자제거

for rule in tqdm(heuristic_rule):
    train_data['document'] = train_data['document'].str.replace(rule[0],rule[1]) # 휴리스틱 규칙적용

train_data['document'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경

train_data.drop_duplicates(subset = ['document'], inplace=True) # document 열에서 중복인 내용이 있다면 중복 제거

train_data = train_data.dropna(how='any') # Null 값 제거

print('전처리 후 테스트용 샘플의 개수 :',len(train_data))


전처리 후 테스트용 샘플의 개수 : 143688


In [9]:
# test_data 전처리

test_data['document'] = test_data['document'].str.replace("[^0-9ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 한글숫자공백외 문자제거

for rule in tqdm(heuristic_rule):
    test_data['document'] = test_data['document'].str.replace(rule[0],rule[1]) # 휴리스틱 규칙적용

test_data['document'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경

test_data.drop_duplicates(subset = ['document'], inplace=True) # document 열에서 중복인 내용이 있다면 중복 제거

test_data = test_data.dropna(how='any') # Null 값 제거

print('전처리 후 테스트용 샘플의 개수 :',len(test_data))


전처리 후 테스트용 샘플의 개수 : 48443


Step6. 띄어쓰기 검사를 실행합니다.
- (주피터 환경) 네이버의 맞춤법 검사 API 호출방식을 이용합니다.
- (코랩 환경) chatspace 띄어쓰기 검사방식을 이용합니다.

In [10]:
# 맞춤법 검사
def correct_sentence(s):
    # API 파라메터 설정
    params = urllib.parse.urlencode({ 
        "_callback": "", 
        "q": s 
    })

    try:
      # 네이버 맞춤법 검사 API 호출
      #data = urllib.request.urlopen("https://m.search.naver.com/p/csearch/ocontent/spellchecker.nhn?" + params) #책의 url을 바꿈
      #data = data.read().decode("utf-8")[1:-2]
      #data = json.loads(data)
      #data = data["message"]["result"]["html"]
      #data = soup = BeautifulSoup(data, "html.parser").getText()
      # 리턴
      data = spacer.space(s) # 구글 코랩에서는 띄어쓰기 교정 chatspace 로 대체실행
      return data
    except HTTPError as e: # 호출 에러 시에는 입력값 그대로 반환
      return s

In [13]:
# 진행상황 체크해가면서 train_data 띄어쓰기 교정
#train_data['correction'] = train_data['document'].progress_apply(correct_sentence)
tempss = correct_sentence(train_data['document'].tolist())
train_data['correction'] = ''
for i in tqdm(range(len(tempss))):
  train_data['correction'].iloc[i] = tempss[i]

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [14]:
train_data # 데이터 확인

,id,document,label,correction
0,9976970,아 더빙 진짜 짜증나네요 목소리,0,아 더빙 진짜 짜증나네요 목소리
1,3819312,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1,흠 포스터 보고 초딩 영화 줄 오버 연기조차 가볍지 않구나
2,10265843,너무재밓었다그래서보는것을추천한다,0,너무 재밓었다 그래서 보는 것을 추천한다
3,9045019,교도소 이야기구먼 솔직히 재미는 없다평점 조정,0,교도소 이야기구먼 솔직히 재미는 없다 평점 조정
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...,1,사이 몬페그의 익살스런 연기가 돋보였던 영화 스파이 더 맨에서 늙어보이기만 했던 커...
...,...,...,...,...
149994,9458520,디케이드 다음에 더블 다음에 오즈인데 더블은 조금밖에 안나오네요,1,디케이드 다음에 더블 다음에 오즈인데 더블은 조금 밖에 안 나오네요
149995,6222902,인간이 문제지 소는 뭔죄인가,0,인간이 문제지 소는 뭔 죄인가
149997,9311800,이게 뭐요 한국인은 거들먹거리고 필리핀 혼혈은 착하다,0,이게 뭐요 한국인은 거들 먹거리고 필리핀 혼혈은 착하다
149998,2376369,청춘 영화의 최고봉방황과 우울했던 날들의 자화상,1,청춘 영화의 최고봉 방황과 우울했던 날들의 자화상


In [15]:
# 진행상황 체크해가면서 test_data 맞춤법 수정
#test_data['correction'] = test_data['document'].progress_apply(correct_sentence)
tempss = correct_sentence(test_data['document'].tolist())
test_data['correction'] = ''
for i in tqdm(range(len(tempss))):
  test_data['correction'].iloc[i] = tempss[i]

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [16]:
test_data # 결과확인

,id,document,label,correction
0,6270596,굳 ㅋ,1,굳 ㅋ
2,8544678,뭐야 이 평점들은 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0,뭐야 이 평점들은 나쁘진 않지만 10점 짜리는 더더욱 아니잖아
3,6825595,지루하지는 않은데 완전 막장임 돈주고 보기에는,0,지루하지는 않은데 완전 막장임 돈 주고 보기에는
4,6723715,3만 아니었어도 별 다섯 개 줬을텐데 왜 3로 나와서 제 심기를 불편하게 하죠,0,3만 아니었어도 별 다섯 개 줬을텐데 왜 3로 나와서 제 심기를 불편하게 하죠
5,7898805,음악이 주가 된 최고의 음악영화,1,음악이 주가 된 최고의 음악 영화
...,...,...,...,...
49995,4608761,오랜만에 평점 로긴했네 ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,1,오랜만에 평점 로긴 했네 ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함
49996,5308387,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따,0,의지 박약들이나 하는 거 다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따
49997,9072549,그림도 좋고 완성도도 높았지만 보는 내내 불안하게 만든다,0,그림도 좋고 완성도도 높았지만 보는 내내 불안하게 만든다
49998,5802125,절대 봐서는 안 될 영화 재미도 없고 기분만 잡치고 한 세트장에서 다 해먹네,0,절대 봐서는 안 될 영화 재미도 없고 기분만 잡치고 한 세트장에서 다 해먹네


Step7. 결과를 저장합니다.

In [17]:
# 전처리 결과저장 ( 이후 훈련 시에 해당파일 사용 )
train_data.to_csv(mypath+'ratings_train_preprocess.csv',sep='|')
test_data.to_csv(mypath+'ratings_test_preprocess.csv',sep='|')